In [15]:
import requests
import json
import os
import pandas as pd
import numpy as np


In [3]:
# Function for converting API response to a table
# Takes the raw response as input
# Assumes that the data is in the `list` item 

def extract_results_table(_txt):
    
    _json = json.loads(_txt)
    _df = pd.DataFrame(_json['list'])

    return _df

In [4]:
payload = {
    'timesort': '1',
    'nohide': '1',
    'checksum': '', 
    'appid': '65328519e78ff0366f242153',
    'token': '0CB822ADFDC6C77C4394',
    'max': '999',
    'catloc': '1',
    'cattotal': '1',
    'units': 'standard',
    'source': 'webtracker'
}


In [17]:
# Build list of URLs

url_list = []    

url_base = 'https://api.rtrt.me/events/ORLANDO-TRIALS-2024/categories/top-{}-marathon/splits/{}M'

for g in ['men', 'women']:
    for i in range(1,27):
        url = url_base.format(g, i)
        url_list.append(url)

    url_list.append('https://api.rtrt.me/events/ORLANDO-TRIALS-2024/categories/top-{}-marathon/splits/FINISH'.format(g))

# Go get results!
    
list_results = []

for url in url_list:
    print(url)
    response = requests.post(url, data=payload)
    response_text = response.text
    response_code = response.status_code

    if response_code == 200:
        df_split_result = extract_results_table(response_text)
        list_results.append(df_split_result)

    else:

        print(response_code)
        print(response_text)  # Prints the response body

df_results = pd.concat(list_results, axis=0)

https://api.rtrt.me/events/ORLANDO-TRIALS-2024/categories/top-men-marathon/splits/1M
https://api.rtrt.me/events/ORLANDO-TRIALS-2024/categories/top-men-marathon/splits/2M
https://api.rtrt.me/events/ORLANDO-TRIALS-2024/categories/top-men-marathon/splits/3M
https://api.rtrt.me/events/ORLANDO-TRIALS-2024/categories/top-men-marathon/splits/4M
https://api.rtrt.me/events/ORLANDO-TRIALS-2024/categories/top-men-marathon/splits/5M
https://api.rtrt.me/events/ORLANDO-TRIALS-2024/categories/top-men-marathon/splits/6M
https://api.rtrt.me/events/ORLANDO-TRIALS-2024/categories/top-men-marathon/splits/7M
https://api.rtrt.me/events/ORLANDO-TRIALS-2024/categories/top-men-marathon/splits/8M
https://api.rtrt.me/events/ORLANDO-TRIALS-2024/categories/top-men-marathon/splits/9M
https://api.rtrt.me/events/ORLANDO-TRIALS-2024/categories/top-men-marathon/splits/10M
https://api.rtrt.me/events/ORLANDO-TRIALS-2024/categories/top-men-marathon/splits/11M
https://api.rtrt.me/events/ORLANDO-TRIALS-2024/categories/top-m

In [18]:
# Do some conversions

df_results['point'] = np.where(df_results['point']=='FINISH', '26.2M', df_results['point'])

df_results['distance'] = df_results['point'].str.slice(0, -1).astype(np.float64)

In [ ]:
# Save the raw results by mile:

df_results[df_results['sex']=='M'].to_csv(os.path.join('..', 'usa-marathon-trials-2024', 'data', 'result_mens_by_mile.csv'))
df_results[df_results['sex']=='F'].to_csv(os.path.join('..', 'usa-marathon-trials-2024', 'data', 'result_womens_by_mile.csv'))

In [21]:
df_results.groupby('distance').agg({'name':'nunique'})
#df_results.groupby('point').agg({'name':'nunique'})

,name
distance,
1.0,348
2.0,348
3.0,348
4.0,348
5.0,348
6.0,348
7.0,346
8.0,346
9.0,344


In [20]:
df_results.groupby('sex').agg({'name':'nunique'})


,name
sex,
F,148
M,200


In [8]:
print(df_results.columns)

Index(['time', 'point', 'label', 'pid', 'bib', 'bib_display', 'tag', 'name',
       'timeOfDay', 'epochTime', 'milePace', 'milePaceAvg', 'mph', 'kmPace',
       'kmPaceAvg', 'kmh', 'etnp', 'etfp', 'startTime', 'sex', 'profile_color',
       '_ver', 'alias', 'pace', 'paceAvg', 'splitTime', 'netTime', 'waveTime',
       'course', 'country', 'country_iso', 'city', 'timestamp', 'i', 'u',
       'place', 'leaderDiff', 'profile_pic', 'team', 'placeChange', 'isFinish',
       'distance'],
      dtype='object')


In [9]:
df_results.pivot(index='name', columns='distance', values='netTime')

distance,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,...,18.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,26.2
name,,,,,,,,,,,,,,,,,,,,,
Aaron Davidson,00:05:01.46,00:10:00.16,00:14:55.19,00:20:01.19,00:25:07.81,00:30:17.11,00:35:37.06,00:40:59.05,00:46:14.35,00:51:54.30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aaron Gruen,00:05:01.06,00:10:01.18,00:15:02.89,00:20:07.85,00:25:16.92,00:30:21.20,00:35:30.32,00:40:38.84,00:45:41.17,00:50:49.33,...,01:32:37.86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Abdi Abdirahman,00:05:00.07,00:09:59.62,00:14:55.89,00:20:00.85,00:25:07.53,00:30:01.41,00:34:59.09,00:40:05.73,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Abinet Adraro,00:05:02.85,00:10:01.97,00:14:57.95,00:20:03.19,00:25:09.04,00:30:04.05,00:35:00.99,00:40:06.63,00:45:10.88,00:50:14.03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Adam Sjolund,00:05:18.83,00:10:28.00,00:15:41.32,00:20:53.69,00:26:10.08,00:31:30.73,00:36:55.07,00:42:16.90,00:47:41.85,00:53:09.75,...,01:39:01.28,01:45:25.64,01:52:04.60,01:58:57.38,02:06:11.11,02:13:17.49,02:19:49.22,02:25:52.04,02:32:27.51,02:33:56.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zachary Hine,00:05:12.60,00:10:21.46,00:15:33.44,00:20:45.66,00:25:59.54,00:31:16.15,00:36:27.54,00:41:37.42,00:46:44.16,00:51:59.65,...,01:34:03.86,01:39:33.16,01:45:10.67,01:51:00.46,01:57:03.25,02:03:26.48,02:10:51.30,02:18:45.56,02:26:41.42,02:28:07.00
Zachary Ornelas,00:05:29.62,00:10:43.62,00:15:57.66,00:21:08.91,00:26:26.30,00:31:43.23,00:36:59.62,00:42:13.22,00:47:24.84,00:52:45.45,...,01:36:40.50,01:42:20.87,01:48:02.62,01:53:56.31,01:59:50.61,02:05:38.37,02:11:32.86,02:17:11.04,02:22:55.53,02:24:07.79
Zachary Ripley,00:05:32.22,00:10:52.65,00:16:14.05,00:21:33.47,00:26:58.20,00:32:20.69,00:37:38.34,00:42:54.80,00:48:08.69,00:53:29.57,...,01:37:59.10,01:44:04.72,01:50:12.96,01:56:49.94,02:04:08.23,02:11:38.93,02:20:00.11,02:28:45.99,02:36:39.42,02:37:47.90


In [14]:
df_results[['name', 'distance', 'place', 'alias', 'pace', 'paceAvg', 'splitTime', 'netTime', 'waveTime']].tail()

,name,distance,place,alias,pace,paceAvg,splitTime,netTime,waveTime
145,Zachary Ripley,26.2,146,FINISH,05:14 min/mile,06:02 min/mile,00:01:08.48,02:37:47.90,02:37:47.90
146,Tyler Sickler,26.2,147,FINISH,05:46 min/mile,06:03 min/mile,00:01:15.60,02:38:35.32,02:38:35.32
147,Jakob Kintzele,26.2,148,FINISH,06:15 min/mile,06:11 min/mile,00:01:21.87,02:41:56.35,02:41:56.35
148,Kevin Colon,26.2,149,FINISH,08:36 min/mile,06:18 min/mile,00:01:52.87,02:45:09.03,02:45:09.03
149,Matthew Rand,26.2,150,FINISH,06:18 min/mile,06:31 min/mile,00:01:22.54,02:50:34.14,02:50:34.14


In [13]:
df_results.dtypes

time              object
point             object
label             object
pid               object
bib               object
bib_display       object
tag               object
name              object
timeOfDay         object
epochTime         object
milePace          object
milePaceAvg       object
mph               object
kmPace            object
kmPaceAvg         object
kmh               object
etnp              object
etfp              object
startTime         object
sex               object
profile_color     object
_ver              object
alias             object
pace              object
paceAvg           object
splitTime         object
netTime           object
waveTime          object
course            object
country           object
country_iso       object
city              object
timestamp         object
i                 object
u                 object
place             object
leaderDiff        object
profile_pic       object
team              object
placeChange       object
